In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L3J1bi9tZWRpYS9hbGYvZGF0b3MvbWlzcmVwb3NpdG9yaW9zL2RvY2VuY2lhL2FwcmVuZGl6YWplLWF1dG9tYXRpY28tcHJhY3RpY2FzLWp1bGlh"
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
function fibonacci(n::Int)
    a = zeros(Float32, n)
    a[1] = 1
    a[2] = 1
    for i in 3:n
        a[i] = a[i-1] + a[i-2]
    end
    return a
end

fibonacci (generic function with 1 method)

In [3]:
fib = fibonacci(30) 
ventana = 4
n = 30 - ventana
# La red esperar una entrada con dimensiones (características, ventana, lote).
X =  Array{Float32}(undef, 1, ventana, n)
Y = Array{Float32}(undef, 1, n)  
for i = 1:n
    serie = fib[i:i+ventana-1]
    etiqueta = fib[i+ventana]
    X[1, : , i] .= serie
    Y[1, i] = etiqueta
end

In [4]:
using Lux, Random

modelo = Chain(
    Recurrence(RNNCell(1 => 8, identity); return_sequence = false),
    Dense(8 => 1)
)

rng = Random.default_rng()
ps, st = Lux.setup(rng, modelo)

((layer_1 = (weight_ih = Float32[-0.00880631; -0.38824496; … ; 0.028445203; -0.93959814;;], weight_hh = Float32[0.93296826 -0.17378621 … 0.018782068 -0.28237218; -0.13712467 0.97534114 … -0.48822352 0.89348274; … ; -0.22203086 -0.43043092 … -0.43872365 0.25599486; -1.4422947 -0.48158586 … -0.74316156 -0.14624931], bias_ih = Float32[0.22773547, -0.9943488, -0.4898884, 0.59630734, 0.46615046, -0.7461217, 0.5620649, 1.0142094], bias_hh = Float32[-2.4209447, -0.72757816, -0.7000604, -0.72231066, 1.4543507, -2.0214987, -0.46558085, -0.13191277]), layer_2 = (weight = Float32[0.3161967 0.5743925 … -0.45958936 -0.48636895], bias = Float32[-0.3252405])), (layer_1 = (rng = TaskLocalRNG(),), layer_2 = NamedTuple()))

In [5]:
using Statistics
function coste(ps, st, X, Y)
    ŷ, _ = modelo(X, ps, st)          # ŷ: (1, batch)
    return mean((ŷ .- Y).^2)
end

coste(ps, st, X, Y)

3.8628536f10

In [6]:
using Optimisers, Zygote
opt = Optimisers.setup(Optimisers.Adam(0.01f0), ps)

nepocas = 200
costes = []
for epoca in 1:nepocas
    # gradiente de la red con respecto a los parámetros
    gs = first(Zygote.gradient(p -> coste(p, st, X, Y), ps))
    opt, ps = Optimisers.update(opt, ps, gs) 
    push!(costes, coste(ps, st, X, Y))
    println("Época $epoca | coste = ", costes[end])
end

Época 1 | coste = 2.4322384e10


Época 2 | coste = 1.2545151e10
Época 3 | coste = 4.209833e9
Época 4 | coste = 2.501131e8
Época 5 | coste = 9.848503e8
Época 6 | coste = 4.674562e9
Época 7 | coste = 7.6489964e9
Época 8 | coste = 7.988745e9
Época 9 | coste = 6.262418e9
Época 10 | coste = 3.7975685e9
Época 11 | coste = 1.6571884e9
Época 12 | coste = 3.7181104e8
Época 13 | coste = 348578.34
Época 14 | coste = 3.0867766e8
Época 15 | coste = 9.568528e8
Época 16 | coste = 1.6341464e9
Época 17 | coste = 2.1255785e9
Época 18 | coste = 2.3253773e9
Época 19 | coste = 2.221723e9
Época 20 | coste = 1.8712984e9
Época 21 | coste = 1.3727585e9
Época 22 | coste = 8.419385e8
Época 23 | coste = 3.8871366e8
Época 24 | coste = 9.556735e7
Época 25 | coste = 14821.585
Época 26 | coste = 8.5490904e7
Época 27 | coste = 2.8637155e8
Época 28 | coste = 5.0951366e8
Época 29 | coste = 6.667808e8
Época 30 | coste = 7.0529984e8
Época 31 | coste = 6.2163955e8
Época 32 | coste = 4.5493334e8
Época 33 | coste = 2.6508627e8
Época 34 | coste = 1.0802152e

In [7]:
using WGLMakie
fig = Figure()
ax = Axis(fig[1, 1], xlabel = "Época", ylabel = "Error cuadrático medio")
lines!(ax, costes)
fig

In [8]:
X_test = reshape(fib[end-ventana : end-1], 1, ventana, 1) 

y_test, _ = modelo(X_test, ps, st)
println("Predicción del término 30: ",  y_test[1, 1])

Predicción del término 30: 832077.5
